## 1) Introduction
This notebook was created as an assignment for an introductory R Analytics course. I hope this may be useful for others learning R like myself. As an fyi, this notbook will take approximately 4.15 hours to complete, and the submission ranked 1718 on the Leaderboard once submitted. Every hour, make sure to touch the notebook as it runs, to not time it out. 

Please note, if you wish to run this, please comment out or delete every section except section #6 for the Submission file. 

To look up the documentation on a function, use a question mark then the function in R.  example:   ?fread

## 2) Preparations

In [ ]:
## Display stats on free memory
gc()

In [ ]:
## Set work directory 
## setwd('/Users/xxx/xxx/')

In [ ]:
##Load Libraries
suppressMessages({
   library(data.table)     # frame data structure
   library(vroom)          # reads rectangular data
   library(tidyverse)      # collection of open source R packages
   library(RcppRoll)       # Provides fast and efficient routines for common rolling / windowed operations
   library(dplyr)          # grammar of data manipulation
   library(lubridate)      # Easier to work w/ dates and time
   library(ggplot2)        # visualization
   library(patchwork)      # combine separate ggplots into the same graphic
   library(scales)         # visualisation
})

If you're working with RStudio, use the below link for instructions on downloading LightGBM - The modelling section below will call the library LightGBM
https://github.com/microsoft/LightGBM/tree/master/R-package#installation

I used this github Online Installer for LightGBM, and followed the approach of downloading Rtools & MinGW for Windows
https://github.com/Laurae2/lgbdl

In [ ]:
sales <- fread("../input/m5-forecasting-accuracy/sales_train_validation.csv", na.strings=c("", "NULL"))  # fread is similar to data.table, but more efficient
prices <- fread("../input/m5-forecasting-accuracy/sell_prices.csv")                                 
cal <- fread("../input/m5-forecasting-accuracy/calendar.csv")

cal2 <- read_csv(str_c('../input/m5-forecasting-accuracy/calendar.csv'), col_types = cols())

In [ ]:
## Free is a function that collects garbage
free <- function() invisible(gc())
free()

## 3) Understanding the data

calendar.csv - Contains information about the dates on which the products are sold.

sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]

sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.

sell_prices.csv - Contains information about the price of the products sold per store and date.

## 4) Overview of the data

In [ ]:
head(cal)
head(prices)
head(sales)

In [ ]:
## Holiday's to predict 
cal[, date  := as.IDate(date)]
d1 <- cal[d == "d_1914", date]
cal[date >= d1][event_name_1 != ""][, c("date","weekday","event_name_1")]
event_in_prediction <- cal[date >= d1][event_name_1 != ""][, event_name_1]

free()

In [ ]:
n_days <- names(sales) %>% str_detect("d_") %>% sum()
ds <- ymd(min(cal$date))
de <-　ds + ddays(n_days-1)

cat(paste0("The number of days in sales_train_validation.csv is ", n_days, ".\n"))
cat(paste0("The range of days is from ", ds, " to ", de, "."))

free()

## 5) Visualization

In [ ]:
sales[, total_sold := apply(sales[, 7:1919], 1, sum)]

options(repr.plot.width = 25, repr.plot.height = 6)

p1 <- sales %>%
  ggplot(aes(x = store_id, y = total_sold, fill = state_id))+
  stat_summary(fun="sum", geom="bar", alpha = 0.7) +
  scale_y_continuous(labels = scales::comma)+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  labs(y = "Total number of items sold", 
       title = "Sales by Store"
       )

p2 <- sales %>%
  ggplot(aes(x = dept_id, y = total_sold, fill = state_id))+
  stat_summary(fun="sum", geom="bar", position = "dodge", alpha = 0.7)+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  scale_y_continuous(labels = scales::comma)+
  labs(y = "Total number of items sold", 
       title = "Sales by Item"
       )

p3 <- 
sales %>% group_by(dept_id) %>% count(item_id) %>% summarise(n_items = length(item_id))%>%
  ggplot(aes(x = dept_id, y = n_items, fill = dept_id))+
  geom_bar(stat = "identity", alpha = 0.7)+
  geom_text(aes(label = n_items), vjust = 1.5, show.legend = F)+
  theme(axis.text.x = element_text(angle = 90, hjust = 1))+
  theme(legend.position = "none")+
  labs(y = "Number of items", 
       title = "Items by Department"
       )
p1+p2+p3  # Uses library patchwork

free()

In [ ]:
# Plot time series graph of total sales
DailySales <- sales[,7:1919]

DailySales <- DailySales %>%
  summarize_all(sum) %>% 
  gather(key="d", value ="sales") %>%
  left_join(cal2[c("d", "date")], by="d") %>%
  mutate(date=as.Date(date)) %>%
  mutate(month_name = month(date, label=TRUE), year = year(date)) %>%
  mutate(month_year = paste(month_name,year,sep="_")) %>%
  mutate(month_name = month(date, label=TRUE), year = year(date)) %>%
  mutate(month_year = paste(month_name,year,sep="_")) %>%
  mutate(wday = wday(date, label=TRUE))

#plot the Time Series
DailySales %>%
  ggplot(aes(date, sales)) +
  geom_line(color = "red") +
  geom_smooth(formula = 'y ~ x', method = "loess", color = "black", span=1/4) +
  scale_y_continuous(labels = scales::label_number_si()) + 
  labs(x = 'Month-Year', y = 'Total Sales', title = '2011-2016 Total Sales') +
  scale_x_date(labels = date_format("%m-%Y"), date_breaks="5 months")

free()

In [ ]:
# Creates a dataframe of monthly sales from 2011-2016 period
MonthSales <- DailySales %>%
  group_by(month_name, year, month_year) %>%
  summarize(sales = sum(sales))

MonthSales %>%
  filter(month_year != 'Jan_2011', month_year != 'Apr_2016') %>% # Remove months that skew the data
  ggplot(aes(x=month_name, y=sales)) +
  geom_boxplot(fill="#4271AE") +
  scale_y_continuous(labels = scales::label_number_si(accuracy=0.01)) +
  labs(x = 'Month', y = 'Total Sales', title = 'Total Sales by Month') +
  theme_bw() 

free()

In [ ]:
## Line Graph
## Calculating the mean average of sales for the month 
mean_MonthSales <- MonthSales %>%
  filter(month_year != 'Jan_2011', month_year != 'Apr_2016') %>% # Remove months that skew the data
  group_by(month_name)  %>%
  summarize(mean_sales= mean(sales)) 

mean_MonthSales %>%
  ggplot(aes(x=month_name, y=mean_sales, group=1)) +
  geom_line(size=1, linetype = "dashed") +
  geom_point() +
  labs(x = 'Month', y = 'Total Sales', title = '2011-2016 Average Sales by Month') 

free()

In [ ]:
#gc()

## 6) Modeling the data

create_dt() function will create a training data table from a wide-format file with leading zeros removed.

create_fea() adds lags, rolling features and time variables to the data table.

In [ ]:
## Load Libraries
library(data.table)  # frame data structure
library(lightgbm)    # gradient boosting framework that uses tree based learning algorithms

set.seed(0)
h <- 28                         # Forecast horizon
max_lags <- 366                 # Number of observations to shift by
tr_last <- 1913                 # Last training day 
fday <- as.IDate("2016-04-25")  # First day to forecast 

#---------------------------
cat("Creating functions...\n")

free <- function() invisible(gc())  # Free is a function that collects garbage

create_dt <- function(is_train = TRUE, nrows = Inf) {
  
  prices <- fread("../input/m5-forecasting-accuracy/sell_prices.csv")
  cal <- fread("../input/m5-forecasting-accuracy/calendar.csv")
  cal[, `:=`(date = as.IDate(date, format="%Y-%m-%d"),  # format date in calendar to yyyy-mm-dd
             is_weekend = as.integer(weekday %chin% c("Saturday", "Sunday")))]   # is_weekend flag - 1 Saturday/ Sunday else 0
  
  if (is_train) {
    dt <- fread("../input/m5-forecasting-accuracy/sales_train_validation.csv", nrows = nrows)
  } else {
    dt <- fread("../input/m5-forecasting-accuracy/sales_train_validation.csv", nrows = nrows,
                drop = paste0("d_", 1:(tr_last-max_lags)))       # keeps only max_lag days from training set
    dt[, paste0("d_", (tr_last+1):(tr_last+2*h)) := NA_real_]    # Empty columns for forecasting
  }

# melt takes data in wide format and stacks a set of columns into a single column of data
  dt <- melt(dt,
             measure.vars = patterns("^d_"),
             variable.name = "d",
             value.name = "sales")
  
  dt <- dt[cal, `:=`(date = i.date, 
                     is_weekend = i.is_weekend,
                     wm_yr_wk = i.wm_yr_wk,
                     event_name_1 = i.event_name_1,
                     snap_CA = i.snap_CA,
                     snap_TX = i.snap_TX,
                     snap_WI = i.snap_WI), on = "d"]
  
  dt[prices, sell_price := i.sell_price, on = c("store_id", "item_id", "wm_yr_wk")]
}

create_fea <- function(dt) {
  
  lag <- c(7, 28, 29)
  dt[, (paste0("lag_", lag)) := shift(.SD, lag), .SDcols = "sales", by = "id"]
  
  win <- c(7, 30, 90, 180)
  dt[, (paste0("roll_mean_28_", win)) := frollmean(lag_28, win, na.rm = TRUE), by = "id"]
  
  win <- c(28)
  dt[, (paste0("roll_max_28_", win)) := frollapply(lag_28, win, max), by = "id"]
  dt[, (paste0("roll_var_28_", win)) := frollapply(lag_28, win, var), by = "id"]
  
  dt[, price_change_1 := sell_price / shift(sell_price) - 1, by = "id"]
  dt[, price_change_365 := sell_price / frollapply(shift(sell_price), 365, max) - 1, by = "id"]
  
  cols <- c("item_id", "state_id", "dept_id", "cat_id", "event_name_1")   
  dt[, (cols) := lapply(.SD, function(x) as.integer(factor(x))), .SDcols = cols]
  
  dt[, `:=`(wday = wday(date),
            mday = mday(date),
            week = week(date),
            month = month(date),
            quarter = quarter(date),
            year = year(date),
            store_id = NULL,
            d = NULL,
            wm_yr_wk = NULL)]
}

#---------------------------
cat("Processing datasets...\n")

tr <- create_dt()   # prepare data for training 
free()              # collect garbage 

create_fea(tr)
free()              # collect garbage 

tr <- na.omit(tr)   # removes NA rows 
y <- tr$sales       # only select sales column from traning set 

idx <- tr[date <= max(date)-h, which = TRUE]   # indices for training 

tr[, c("id", "sales", "date") := NULL]  # remove id, sales, date columns
free()                                  # collect garbage 

tr <- data.matrix(tr)  # returns a matrix by converting all variables to numeric 
free()                 # collect garbage 

cats <- c("item_id", "state_id", "dept_id", "cat_id",
          "wday", "mday", "week", "month", "quarter", "year", "is_weekend",
          "snap_CA", "snap_TX", "snap_WI")   # list categorical features

xtr <- lgb.Dataset(tr[idx, ], label = y[idx], categorical_feature = cats)     # construct lgb dataset 
xval <- lgb.Dataset(tr[-idx, ], label = y[-idx], categorical_feature = cats)  # construct lgb dataset

rm(tr, y, idx)
free()            # collect garbage 

#---------------------------
cat("Training model...\n")

# Poisson traning model, metirc of root mean squared error used 
p <- list(objective = "poisson",
          metric ="rmse",
          force_row_wise = TRUE,
          learning_rate = 0.075,
          sub_feature = 0.8,
          sub_row = 0.75,
          bagging_freq = 1,
          lambda_l2 = 0.1,
          nthread = 4)

m_lgb <- lgb.train(params = p,
                   data = xtr,
                   nrounds = 2000,
                   valids = list(valid = xval),
                   early_stopping_rounds = 400,
                   eval_freq = 200)

cat("Best score:", m_lgb$best_score, "at", m_lgb$best_iter, "iteration")                         
lgb.plot.importance(lgb.importance(m_lgb), 20)

rm(xtr, xval, p)
free()

#---------------------------
cat("Forecasting...\n") 

te <- create_dt(FALSE)

for (day in as.list(seq(fday, length.out = 2*h, by = "day"))){
  cat(as.character(day), " ")
  tst <- te[date >= day - max_lags & date <= day]
  create_fea(tst)
  tst <- data.matrix(tst[date == day][, c("id", "sales", "date") := NULL])
  te[date == day, sales := predict(m_lgb, tst)]
}

te[date >= fday
   ][date >= fday+h, id := sub("validation", "evaluation", id)
     ][, d := paste0("F", 1:28), by = id
       ][, dcast(.SD, id ~ d, value.var = "sales")
         ][, fwrite(.SD, "Submission_LBGM.csv")]